In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
!pip install ipyplot
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

In [ ]:
#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install tridentx --upgrade

In [ ]:
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import cv2
import random
from shutil import copyfile
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
src_dataset = '../input/histopathologic-cancer-detection'
train_data_dir = 'dataset/train'
test_data_dir = 'dataset/test'
validation_split = 0.2

batch_size = 32
img_height = 96 #96 original size
img_width = 96

In [ ]:
def preprocess_dataset():
    print('dataset re-structure...')
    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)
            
    for file in tqdm(os.listdir( os.path.join(src_dataset, 'train'))):
        filename, file_extension = os.path.splitext(file)
        if file_extension == '.tif':
            file_path = os.path.join(src_dataset, 'train', file)
            new_img_path = os.path.join(train_data_dir, filename+'.jpg')
            cv2.imwrite(new_img_path, cv2.imread(file_path)  )

def get_combined(img, ccrop=(16,32,64,96)):
    width = img.shape[1]
    height = img.shape[0]
    
    cimgs = []
    for c in ccrop:
        x1 = int(width/2 - c/2)
        y1 = int(height/2 - c/2)
        x2 = x1 + c
        y2 = y1 + c
        center_img = img[y1:y2, x1:x2]
        center_img = cv2.resize(center_img, (48,48))
        cimgs.append(center_img)
        
    vis1 = np.concatenate((cimgs[0], cimgs[1]), axis=1)
    vis2 = np.concatenate((cimgs[2], cimgs[3]), axis=1)
    vis = np.concatenate((vis1, vis2), axis=0)
    
    return vis

def get_combined_B(img, ccrop=(32,32)):
    width = img.shape[1]
    height = img.shape[0]
    x1 = int(width/2 - ccrop[0]/2)
    y1 = int(height/2 - ccrop[1]/2)
    x2 = x1 + ccrop[0]
    y2 = y1 + ccrop[1]
    center_img = cv2.resize(img[y1:y2, x1:x2], (48,96))
    img = cv2.resize(img, (48,96))
    vis = np.concatenate((center_img, img), axis=1)
    return vis

def preprocess_image():
    for classname in ['0', '1']:
        print('image pre-processt, class' ,classname)
        folder_path = os.path.join('dataset/train/', classname)
        for file in tqdm(os.listdir(folder_path)):
            filename, file_extension = os.path.splitext(file)
            if file_extension == '.jpg':
                file_path = os.path.join(folder_path, file)
                img = get_combined( cv2.imread(file_path) )
                cv2.imwrite( file_path, img )

def convert_test_ds():
    src_folder = os.path.join(src_dataset, 'test')
    folder = 'dataset/test'
    for file in tqdm(os.listdir(src_folder)):
            filename, file_extension = os.path.splitext(file)
            if file_extension == '.tif':
                src_file_path = os.path.join(src_folder, file)
                file_path = os.path.join(folder, file)
                img = cv2.imread(src_file_path)
                img = get_combined(img, ccrop=(16,32,64,96))
                cv2.imwrite( os.path.join(folder, filename+'.jpg'), img )
                #os.remove(file_path)

In [ ]:
#for first time only
preprocess_dataset()
preprocess_image()
convert_test_ds()

In [ ]:
train_data_dir = pathlib.Path(train_data_dir)
test_data_dir = pathlib.Path(test_data_dir)

image_count = len(list(train_data_dir.glob('0/*.jpg')))
print("0 class count of training dataset:", image_count)
image_count = len(list(train_data_dir.glob('1/*.jpg')))
print("1 class count of training dataset:", image_count)

train_img_list = list(train_data_dir.glob('0/*.jpg'))
print(train_img_list[1])
img = cv2.imread( str(train_img_list[1]) )
print('images shape is', img.shape)

In [ ]:
import glob
#透過glob所全部train資料夾中所有可用圖片
imgs=glob.glob( os.path.join(train_data_dir, '*.jpg') )
print(len(imgs))

#ImageDatset(imgs,symbol='image')
f=open( os.path.join(src_dataset, 'train_labels.csv'),'r',encoding='utf-8-sig')
rows=f.readlines()
print(rows[:3])
image_path=[]
labels=[]
test_image_path=[]
test_labels=[]

rows=rows[1:] #拿掉第一筆標頭
random.shuffle(rows)#隨機洗牌
for row in rows:
    cols=row.strip().split(',') #移除\n然後逗號分割
    if random.random()<=0.3:
        test_image_path.append( os.path.join(train_data_dir,'.jpg'.format(cols[0])))
        test_labels.append(int(cols[1]))
    else:
        image_path.append(os.path.join(train_data_dir,'.jpg'.format(cols[0])))
        labels.append(int(cols[1]))
print(len(image_path))
print(len(labels))
print(len(test_image_path))
print(len(test_labels))

In [ ]:
#資料集
ds1=ImageDataset(image_path,symbol='image')
ds2=LabelDataset(labels,symbol='label')

ds1_t=ImageDataset(test_image_path,symbol='image')
ds2_t=LabelDataset(test_labels,symbol='label')

#與Iterator構成data provider
data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2),testdata=Iterator(data=ds1_t,label=ds2_t))

#設定DataProvider的預處理流程
data_provider.image_transform_funcs=[Normalize(127.5,127.5)]

#即可完成設定，可以透過next()來確認數據是否正常拋出，以及是否有正確產生輸出數據的signature
img_data,label_data=data_provider.next()
print(data_provider.signature)
print(img_data.shape)
print(label_data.shape)

data_provider.image_transform_funcs=[
    auto_level(),
    RandomAdjustGamma(scale=(0.6,1.4)),#調整明暗
    RandomAdjustHue(scale=(-0.5,0.5)),#調整色相
    RandomAdjustSaturation(scale=(0.6,1.4)),#調整飽和度
    SaltPepperNoise(0.05),#加入胡椒鹽噪音
    RandomErasing(), #加入隨機擦去
    Normalize(127.5,127.5)] #標準化

#測試集數據不需要做數據增強
data_provider.testdata.data.image_transform_funcs=[
    Normalize(127.5,127.5)] #標準化

img_data,label_data=data_provider.next()
print(img_data.shape)
print(label_data.shape)
test_img_data,test_label_data=data_provider.next_test()
print(test_img_data.shape)
print(test_label_data.shape)
data_provider.preview_images()

In [ ]:
print(val_batches)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Augmentation

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomFlip('vertical'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(1.0),
  tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.2)
])
'''
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.5)
])
'''

In [ ]:
#preview for augmentations
for image, _ in train_ds.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

In [ ]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (img_height,img_width) + (3,)

basemodel = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = basemodel(image_batch)
print(feature_batch.shape)

In [ ]:
basemodel.trainable = False
basemodel.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = rescale(x)
x = preprocess_input(x)
x = basemodel(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0005
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
loss0, accuracy0 = model.evaluate(val_ds)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
initial_epochs = 30
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

In [ ]:
initial_epochs = history.epoch[-1]  #continued from last
add_epochs = 20
total_epochs =  initial_epochs + add_epochs + 1

acc += history.history['accuracy']
val_acc += history.history['val_accuracy']

loss += history.history['loss']
val_loss += history.history['val_loss']

history = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=initial_epochs,
                         validation_data=val_ds)

In [ ]:
#Load images list, and load labels
import glob
#透過glob所全部train資料夾中所有可用圖片
imgs=glob.glob('../input/histopathologic-cancer-detection/train/*.tif')
print(len(imgs))

#ImageDatset(imgs,symbol='image')
f=open('../input/histopathologic-cancer-detection/train_labels.csv','r',encoding='utf-8-sig')
rows=f.readlines()
print(rows[:3])

rows=rows[1:] #拿掉第一筆標頭
random.shuffle(rows)#隨機洗牌

In [ ]:
#Split to training & testing dataset

image_path=[]
labels=[]
test_image_path=[]
test_labels=[]

for row in rows:
    cols=row.strip().split(',') #移除\n然後逗號分割
    if random.random()<=0.3:
        test_image_path.append('../input/histopathologic-cancer-detection/train/{0}.tif'.format(cols[0]))
        test_labels.append(int(cols[1]))
    else:
        image_path.append('../input/histopathologic-cancer-detection/train/{0}.tif'.format(cols[0]))
        labels.append(int(cols[1]))
print('training dataset imgages / labels', len(image_path), '/', len(labels))

print('testing dataset images / labels', len(test_image_path), '/', len(test_labels))


In [ ]:
#將一個個放置於image_path, labels及test_image_path, test_labels的列表，透過Trident提供的ImageDataset和LabelDataset載入。
ds1_tmp=ImageDataset(image_path,symbol='image')
ds2=LabelDataset(labels,symbol='label')

ds1_t=ImageDataset(test_image_path,symbol='image')
ds2_t=LabelDataset(test_labels,symbol='label')


In [ ]:
#let's preview some images
#!pip install ipyplot
import ipyplot
import cv2
from tqdm import tqdm

def get_combined(img, ccrop=(32,32)):
    width = img.shape[1]
    height = img.shape[0]
    x1 = int(width/2 - ccrop[0]/2)
    y1 = int(height/2 - ccrop[1]/2)
    x2 = x1 + ccrop[0]
    y2 = y1 + ccrop[1]
    center_img = cv2.resize(img[y1:y2, x1:x2], (48,96))
    img = cv2.resize(img, (48,96))
    vis = np.concatenate((center_img, img), axis=1)
    return vis


ds1 = []
for img in tqdm(ds1_tmp):
    img_c = get_combined(img, (32,32))
    ds1.append(img_c.copy())

print(ds1[3].shape)
ipyplot.plot_images([ds1[3]], img_width=250)


In [ ]:
import torchvision, torch, torchaudio, matplotlib
print(torchvision.__version__)
print(torch.__version__)
print(torchaudio.__version__)
print(matplotlib.__version__)

In [ ]:
#與Iterator構成data provider
#設定Trident Data provider，很類似Keras的Data generator，但使用起來更為方便。
data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2),testdata=Iterator(data=ds1_t,label=ds2_t))

#即可完成設定，可以透過next()來確認數據是否正常拋出，以及是否有正確產生輸出數據的signature
img_data,label_data=data_provider.next()
print(data_provider.signature)
print(img_data.shape)
print(label_data.shape)

#設定DataProvider的預處理流程
data_provider.image_transform_funcs=[
    auto_level(),
    RandomAdjustGamma(scale=(0.6,1.4)),#調整明暗
    RandomAdjustHue(scale=(-0.5,0.5)),#調整色相
    RandomAdjustSaturation(scale=(0.6,1.4)),#調整飽和度
    SaltPepperNoise(0.05),#加入胡椒鹽噪音
    RandomErasing(), #加入隨機擦去
    Normalize(127.5,127.5)] #標準化

img_data,label_data=data_provider.next()
print(img_data.shape)
print(label_data.shape)

data_provider.preview_images()

In [ ]:
#測試集數據不需要做數據增強
data_provider.testdata.data.image_transform_funcs=[
    Normalize(127.5,127.5)] #標準化

test_img_data,test_label_data=data_provider.next_test()
print(test_img_data.shape)
print(test_label_data.shape)

data_provider.preview_images()

In [ ]:
from trident.models import efficientnet,densenet,resnet
net1=efficientnet.EfficientNetB0(pretrained=True,include_top=True,freeze_features=True,input_shape=(3,96,96),classes=2)
net1.model[-1].add_noise=True
net1.model[-1].noise_intensity=0.12

In [ ]:
class CustomCropFlatten(Layer):
    """
  
    """
    def __init__(self,name='CustomCropFlatten'):
        super(CustomCropFlatten, self).__init__()
        self.name = name

    def forward(self, x, **kwargs):
        #(None, 112, 6, 6)
        #target_area=2x2
        target=x[:,:,2:4,2:4]
        B,C,H,W=int_shape(target)
        outside1=reduce_max(x[:,:,1,1:5],axis=-1)
        outside2=reduce_max(x[:,:,4,1:5],axis=-1)
        outside3=reduce_max(x[:,:,1:5,1],axis=-1)
        outside4=reduce_max(x[:,:,1:5,4],axis=-1)
        outside=stack([outside1,outside2,outside3,outside4],axis=-1)
        target=reshape(target,(B,C,-1))
        return reshape(concate([target,outside],axis=-1),(B,-1))

net2=efficientnet.EfficientNetB0(pretrained=True,include_top=False,freeze_features=True,input_shape=(3,96,96),classes=2)
net2.model.remove_at(-1)
net2.model.remove_at(-1)
net2.model.remove_at(-1)
net2.model.remove_at(-1)
net2.model.remove_at(-1)
net2.model.remove_at(-1)
net2.model.add_module('custom',CustomCropFlatten())
net2.model.add_module('fc',Dense(2,activation=None))
net2.model.add_module('softmax',SoftMax(-1,add_noise=True,noise_intensity=0.12))
net2.summary()

In [ ]:
net3=resnet.ResNet50(pretrained=True,include_top=False,freeze_features=True,input_shape=(3,96,96),classes=2)
net3.model.remove_at(-1)
net3.model.add_module('custom',CustomCropFlatten())
net3.model.add_module('fc',Dense(2,activation=None))
net3.model.add_module('softmax',SoftMax(axis=-1,add_noise=True,noise_intensity=0.12))
net3.summary()

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
# make a prediction

def auc(output,target):
    
    output_np=to_numpy(exp(output))[:,1]
    target_np=to_numpy(target)
    return roc_auc_score(target_np, output_np)

def punishment(output,target):
    mask=target==1
    mask_neg=target==0
    masked_positive=exp(output)[:,1][mask]
    masked_negative=exp(output)[:,1][mask_neg]
    return 1-(masked_positive.mean()-masked_negative.mean())

#challenger1 使用DiffGrad優化器、累積梯度
net1.with_optimizer(DiffGrad,lr=1e-3,gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True,label_smooth=True))\
.with_loss(F1ScoreLoss,loss_weight=0.5)\
.with_loss(punishment,loss_weight=0.2)\
.with_metric(accuracy,ignore_index=0)\
.with_metric(recall,ignore_index=0)\
.with_metric(auc)\
.with_regularizer('l2',1e-5)\
.with_model_save_path('./Models/eff_1.pth')\
.with_accumulate_grads(5)\
.with_callbacks(CosineLR(max_lr=1e-3, min_lr=1e-8,period=5000))\
.unfreeze_model_scheduling(300,unit='batch',module_name='block7a')\
.with_automatic_mixed_precision_training()

#challenger1 使用DiffGrad優化器、累積梯度
net2.with_optimizer(DiffGrad,lr=1e-3,gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True,label_smooth=True))\
.with_loss(F1ScoreLoss,loss_weight=0.5)\
.with_loss(punishment,loss_weight=0.2)\
.with_metric(accuracy,ignore_index=0)\
.with_metric(recall,ignore_index=0)\
.with_metric(auc)\
.with_regularizer('l2',1e-5)\
.with_model_save_path('./Models/eff_2.pth')\
.with_accumulate_grads(10)\
.with_callbacks(CosineLR(max_lr=1e-3, min_lr=1e-8,period=5000))\
.unfreeze_model_scheduling(300,unit='batch',module_name='block5c')\
.with_automatic_mixed_precision_training()



#challenger1 使用DiffGrad優化器、累積梯度
net3.with_optimizer(DiffGrad,lr=1e-3,gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True,label_smooth=True))\
.with_loss(F1ScoreLoss,loss_weight=0.5)\
.with_loss(punishment,loss_weight=0.2)\
.with_metric(accuracy,ignore_index=0)\
.with_metric(recall,ignore_index=0)\
.with_metric(auc)\
.with_regularizer('l2',1e-5)\
.with_model_save_path('./Models/resnet_1.pth')\
.with_accumulate_grads(10)\
.with_callbacks(CosineLR(max_lr=1e-3, min_lr=1e-8,period=5000))\
.unfreeze_model_scheduling(300,unit='batch',module_name='layer3.5')\
.with_automatic_mixed_precision_training()

In [ ]:
#使用outsample驗證
def draw_roc(training_context): #建模訓練階段都靠training_context通訊，傳遞所有需要的訊息
    if training_context['steps']==10 or (training_context['steps']+1)%100==0:
        model=copy.deepcopy(training_context['current_model'])#複製一份模型
        model.eval()  #切換為推論模式
        traindata=training_context['train_data'] #取出測試數據
        input_data=traindata['image']
        target_data=traindata['label']
        
        output=model(input_data)
        target_np=to_numpy(target_data)
        output_np=to_numpy(output) #推論階段softmax就會是一般softmax(訓練階段是log_softmax)，所以不必再做處理，而且所有noise, dropout都會消失
        fpr, tpr,_=roc_curve(target_np, output_np[:,1])
        plt.figure(1)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr, tpr, label='area = {:.3f}'.format(roc_auc_score(target_np, output_np[:,1])))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()
        
net1.trigger_when('on_batch_end', frequency=1, action=draw_roc)
net2.trigger_when('on_batch_end', frequency=1, action=draw_roc)
net3.trigger_when('on_batch_end', frequency=1, action=draw_roc)

In [ ]:
!pip install tensorboard

In [ ]:
plan=TrainingPlan()\
    .add_training_item(net1,name='net1')\
    .add_training_item(net2,name='net2')\
    .add_training_item(net3,name='net3')\
    .with_data_loader(data_provider)\
    .with_batch_size(128)\
    .repeat_epochs(5)\
    .out_sample_evaluation_scheduling(100)\
    .print_gradients_scheduling(100,unit='batch')\
    .print_progress_scheduling(10,unit='batch')\
    .display_loss_metric_curve_scheduling(200)\
    .save_model_scheduling(20,unit='batch')
    #.with_tensorboard()

plan.start_now(collect_data_inteval=10)